# 3. Feature Enginerring

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os 
import sys
sys.path.append(os.path.join(os.path.abspath('..')))

In [3]:
import src.data_loading as dl
import scripts.feature_enigneering as fe

In [4]:
output_data = '../data/processed'
df = pd.read_csv(output_data + '/processed_data.csv')
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15T02:44:21Z,2,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15T03:34:21Z,2,0


1. Create Aggregate Features

In [5]:
# Total transaction amount 
df['TotalTransactionAmount'] = df.groupby('CustomerId')['Value'].transform('sum')

In [6]:
# df['TotalT'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().sum()))

In [7]:
# Average Transaction Amount
df['AverageTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().mean()))

In [8]:
# df['ava'] = df.groupby('CustomerId')['Value'].transform('mean')

In [9]:
# Transaction Count
df['TransationCount'] = df.groupby('CustomerId')['TransactionId'].transform('size')

Note that the std function will calculate the population standard deviation by default. So i want to calculate the sample standard deviation, so i can use the ddof parameter:

In [10]:
# Standard Deviation of Transaction Amounts
df['StdTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().std(ddof=1)))

In [11]:
# df['StdDevTransaction'] = df.groupby('CustomerId')['Value'].transform('std')

In [12]:
#categorical and numerical columns into variables
num_col = df.select_dtypes(include=np.number).columns
cat_col = df.select_dtypes(exclude=np.number).columns

## 2. Extract Features

In [13]:
# Apply the function
fe.extract_features(df, ['TransactionStartTime'])

In [14]:
pd.set_option('display.max_columns', None)
df.sample(6)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TransationCount,StdTransactionAmount,TransactionStartTime_Month,TransactionStartTime_Week,TransactionStartTime_Day,TransactionStartTime_Dayofweek,TransactionStartTime_Dayofyear,TransactionStartTime_Is_month_end,TransactionStartTime_Is_month_start,TransactionStartTime_Is_quarter_end,TransactionStartTime_Is_quarter_start,TransactionStartTime_Is_year_end,TransactionStartTime_Is_year_start,TransactionStartTime_Hour,TransactionStartTime_Minute,TransactionStartTime_Second,TransactionStartTime_Elapsed
87327,54076,39620,4670,3642,5138,UGX,256,6,3,airtime,3,2000.0,2000,2019-02-06 19:01:07,2,0,2257225,7270.746753,308,1.240273e+04,2,6,6,2,37,False,False,False,False,False,False,19,1,7,1549479667
79407,64702,45452,572,2950,909,UGX,256,5,15,financial_services,3,230000.0,230000,2019-01-31 14:52:21,2,0,83480760,725919.652174,115,1.297134e+06,1,5,31,3,31,True,False,False,False,False,False,14,52,21,1548946341
89401,20310,66243,4840,3829,5196,UGX,256,4,6,financial_services,2,-1000.0,1000,2019-02-08 09:47:52,2,0,32000,5333.333333,6,3.945462e+03,2,6,8,4,39,False,False,False,False,False,False,9,47,52,1549619272
48502,35278,9251,318,3087,647,UGX,256,6,3,airtime,3,1000.0,1000,2019-01-01 13:25:01,2,0,3905264,2076.180845,1869,7.680888e+03,1,1,1,1,1,False,True,False,True,False,True,13,25,1,1546349101
16431,98447,137393,4249,4429,7343,UGX,256,4,3,airtime,2,-25000.0,25000,2018-12-04 16:54:15,4,0,104900000,25641.652408,4091,1.295260e+04,12,49,4,1,338,False,False,False,False,False,False,16,54,15,1543942455
10945,67369,22573,4841,3829,3882,UGX,256,4,6,financial_services,2,-26.0,26,2018-11-30 04:19:32,2,0,446922,1655.865672,268,4.404733e+03,11,48,30,4,334,True,False,False,False,False,False,4,19,32,1543551572


In [15]:
# Boolean to int 
df[df.select_dtypes(include='bool').columns] = df.select_dtypes(include='bool').astype(int)

## Encode Categorical Variables

In [16]:
# Frequency Encoding
fe.encode_FE(df, ['CustomerId', 'TransactionId'])

CustomerId_FE , TransactionId_FE , 

In [22]:
# Label Encoding
fe.encode_LE(df, ['ProductCategory', 'ProductId', 'SubscriptionId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ChannelId', 'PricingStrategy'])

ProductCategory , ProductId , SubscriptionId , CurrencyCode , CountryCode , ProviderId , ChannelId , PricingStrategy , 

In [30]:
# Group Aggregation
fe.encode_AG(df, 'CustomerId', ['Value'], ['mean','min','max','std'])

CustomerId_Value_mean , CustomerId_Value_min , CustomerId_Value_max , CustomerId_Value_std , 

In [24]:
# Combine Features
fe.encode_CB(df, 'ProductId', 'ProductCategory')

ProductId_ProductCategory , 

In [25]:
# Group Aggregation NUnique
fe.encode_AG2(df, 'CustomerId', ['TransactionId'])

CustomerId_TransactionId_ct , 

In [31]:
pd.set_option('display.max_columns', None)
df.sample(6)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TransationCount,StdTransactionAmount,TransactionStartTime_Month,TransactionStartTime_Week,TransactionStartTime_Day,TransactionStartTime_Dayofweek,TransactionStartTime_Dayofyear,TransactionStartTime_Is_month_end,TransactionStartTime_Is_month_start,TransactionStartTime_Is_quarter_end,TransactionStartTime_Is_quarter_start,TransactionStartTime_Is_year_end,TransactionStartTime_Is_year_start,TransactionStartTime_Hour,TransactionStartTime_Minute,TransactionStartTime_Second,TransactionStartTime_Elapsed,CustomerId_FE,TransactionId_FE,ProductId_ProductCategory,CustomerId_TransactionId_ct,CustomerId_Value_mean,CustomerId_Value_min,CustomerId_Value_max,CustomerId_Value_std
28198,85805,47863,3441,2329,3876,0,0,5,2,0,2,1000.0,1000,2018-12-15 10:36:51,2,0,492309,1289.000000,381,3393.540475,12,50,15,5,349,0,0,0,0,0,0,10,36,51,1544870211,762.0,2.0,2_0,381.0,1292.149606,6,27500,3402.192429
94861,98661,26720,4841,2867,4840,0,0,3,5,2,1,-50.0,50,2019-02-12 14:53:52,2,0,1024812,2351.156221,434,7465.310560,2,7,12,1,43,0,0,0,0,0,0,14,53,52,1549983232,868.0,2.0,5_2,434.0,2361.317972,15,100000,7480.261373
77302,6228,10464,4841,2867,858,0,0,3,5,2,1,-25.0,25,2019-01-29 12:37:55,2,0,28871,2293.416667,12,2865.936638,1,5,29,1,29,0,0,0,0,0,0,12,37,55,1548765475,24.0,2.0,5_2,12.0,2405.916667,25,10000,2891.063361
73048,38064,93002,1742,772,2130,0,0,5,16,7,2,3500.0,3500,2019-01-25 18:19:37,2,0,110952,3774.206897,29,7519.661720,1,4,25,4,25,0,0,0,0,0,0,18,19,37,1548440377,58.0,2.0,16_7,29.0,3825.931034,24,30000,7530.873455
42269,88721,33669,2843,458,3265,0,0,5,2,0,2,1500.0,1500,2018-12-27 13:14:03,2,0,446198,1379.092105,323,5202.621691,12,52,27,3,361,0,0,0,0,0,0,13,14,3,1545916443,646.0,2.0,2_0,323.0,1381.417957,10,50223,5204.408799
49151,81192,18634,2073,860,2477,0,0,5,3,0,2,1000.0,1000,2019-01-02 14:38:52,2,0,676978,2145.669355,310,5308.432382,1,1,2,2,2,0,0,0,0,0,0,14,38,52,1546439932,620.0,2.0,3_0,310.0,2183.800000,10,47000,5369.767920


In [29]:
df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'TotalTransactionAmount', 'AverageTransactionAmount', 'TransationCount',
       'StdTransactionAmount', 'TransactionStartTime_Month',
       'TransactionStartTime_Week', 'TransactionStartTime_Day',
       'TransactionStartTime_Dayofweek', 'TransactionStartTime_Dayofyear',
       'TransactionStartTime_Is_month_end',
       'TransactionStartTime_Is_month_start',
       'TransactionStartTime_Is_quarter_end',
       'TransactionStartTime_Is_quarter_start',
       'TransactionStartTime_Is_year_end',
       'TransactionStartTime_Is_year_start', 'TransactionStartTime_Hour',
       'TransactionStartTime_Minute', 'TransactionStartTime_Second',
       'TransactionStartTime_Elapsed', 'CustomerId_FE', 'TransactionId_FE',
      